# Data Cleaning Listado de Listado de Beneficiarios 2019

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_19 -> Dataset con el listado de TODOS los beneficiarios de 2019.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2019

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_19 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/fertilizantes_2019.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_19 = beneficiarios_19[~((beneficiarios_19['ENTIDAD'] == 'NACIONAL') & (beneficiarios_19['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_19['LOCALIDAD'] == 'NACIONAL'))]

In [5]:
beneficiarios_19.shape

(275026, 16)

In [6]:
beneficiarios_19['ENTIDAD'].unique()

array(['GUERRERO', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [7]:
beneficiarios_19.dropna(inplace=True)

In [8]:
# Obtenemos las localidades únicas en el dataset.
Municipios_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO']]
Municipios_19 = Municipios_19.drop_duplicates()

In [9]:
# Estandarizamos la limpieza de los datos
Municipios_19['ENTIDAD_c_benef'] = Municipios_19['ENTIDAD'].apply(clean_text)
Municipios_19['MUNICIPIO_c_benef'] = Municipios_19['MUNICIPIO'].apply(clean_text)

In [10]:
# Creamos las dos keys de beneficiarios 
Municipios_19['KEY_benef_mun'] = Municipios_19['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_19[
    'MUNICIPIO_c_benef'].astype(str)

In [11]:
Municipios_19.shape

(81, 5)

In [12]:
Municipios_19.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2019

Obtener listado único de municipios y listado único de localidades de inegi de 2019 con sus claves.

In [13]:
path_dataset_inegi_2019 = '../../data/inegi/dataset_inegi_clean_2019.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2019 = pd.read_csv(path_dataset_inegi_2019)

### 1.2.1 INEGI 2019 Municipios únicos para cada año.

In [14]:
dataset_inegi_2019['KEY_inegi_municipio'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2019['KEY_inegi_localidad'] = dataset_inegi_2019['Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2019['Localidad_c_inegi'].astype(str)

In [15]:
INEGI_UNIQUEMUN_2019 = dataset_inegi_2019.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad", "POB_TOTAL"])

INEGI_UNIQUEMUN_2019 = INEGI_UNIQUEMUN_2019.drop_duplicates()

In [16]:
INEGI_UNIQUEMUN_2019.shape

(2465, 7)

In [17]:
INEGI_UNIQUEMUN_2019.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

In [18]:
INEGI_UNIQUEMUN_2019.drop_duplicates(subset='KEY_inegi_municipio', keep='first', inplace=True)

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2019, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_19 y el dataset de INEGI correspondiente.

In [19]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [20]:
diccionario_MUN_19 = fuzzy_merge_benef2019_2022(Municipios_19, INEGI_UNIQUEMUN_2019, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_19.shape

(81, 14)

In [21]:
diccionario_MUN_19.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)
diccionario_MUN_19.drop_duplicates(subset='KEY_benef_mun', keep='first', inplace=True)

In [22]:
diccionario_MUN_19.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19.csv', index=False)

In [23]:
diccionario_MUN_19_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19_simple.csv')

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [24]:
beneficiarios_19.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [25]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_19['ESTADO_Clean'] = beneficiarios_19['ENTIDAD'].apply(clean_text)
beneficiarios_19['MUNICIPIO_Clean'] = beneficiarios_19['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_19['Estado-mun-KEY'] = beneficiarios_19['ESTADO_Clean'].astype(str) + '-' + beneficiarios_19[
    'MUNICIPIO_Clean'].astype(str)

In [26]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_19, diccionario_MUN_19_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [27]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [28]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2019, left_on="KEY_inegi_municipio",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [29]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,MUNICIPIO_Clean,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi
0,TORRES ROMERO SANTANA,CENTRO PAÍS,GUERRERO,AZOYÚ,LOS CHEGÜES,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,azoyu,guerrero-azoyu,guerrero-azoyu,guerrero-azoyu,12,Guerrero,13,Azoyú,guerrero,azoyu
1,GOMEZ PADILLA DEMETRIA,CENTRO PAÍS,GUERRERO,TECOANAPA,MECATEPEC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa,guerrero,tecoanapa
2,GUERRERO MELCHOR ADRIAN,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TLAJOCOTLA (LA HACIENDA),Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,teloloapan,guerrero-teloloapan,guerrero-teloloapan,guerrero-teloloapan,12,Guerrero,58,Teloloapan,guerrero,teloloapan
3,ROMERO GONZALEZ CELERINA,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TELOLOAPAN,Medio,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,teloloapan,guerrero-teloloapan,guerrero-teloloapan,guerrero-teloloapan,12,Guerrero,58,Teloloapan,guerrero,teloloapan
4,GOMEZ CERON MA.MIREYA,CENTRO PAÍS,GUERRERO,TECOANAPA,LOS SAUCES,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa,guerrero,tecoanapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275018,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón,guerrero,taxco de alarcon
275019,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón,guerrero,taxco de alarcon
275020,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,GUERRERO,COCULA,COCULA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,cocula,guerrero-cocula,guerrero-cocula,guerrero-cocula,12,Guerrero,17,Cocula,guerrero,cocula
275021,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,GUERRERO,ATLIXTAC,XALPITZÁHUAC,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,atlixtac,guerrero-atlixtac,guerrero-atlixtac,guerrero-atlixtac,12,Guerrero,10,Atlixtac,guerrero,atlixtac


In [30]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio', 'CVE_ENT', 'Entidad_inegi',
       'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi'],
      dtype='object')

In [31]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'KEY_inegi_municipio', 'Entidad_c_inegi','Municipio_c_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [32]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,TORRES ROMERO SANTANA,CENTRO PAÍS,GUERRERO,AZOYÚ,LOS CHEGÜES,Cultivo de maíz grano,30/06/2019,2430.55,PV2019,guerrero-azoyu,12,Guerrero,13,Azoyú
1,GOMEZ PADILLA DEMETRIA,CENTRO PAÍS,GUERRERO,TECOANAPA,MECATEPEC,Cultivo de maíz grano,02/07/2019,4383.10,PV2019,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa
2,GUERRERO MELCHOR ADRIAN,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TLAJOCOTLA (LA HACIENDA),Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-teloloapan,12,Guerrero,58,Teloloapan
3,ROMERO GONZALEZ CELERINA,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TELOLOAPAN,Cultivo de maíz grano,17/07/2019,4383.10,PV2019,guerrero-teloloapan,12,Guerrero,58,Teloloapan
4,GOMEZ CERON MA.MIREYA,CENTRO PAÍS,GUERRERO,TECOANAPA,LOS SAUCES,Cultivo de maíz grano,18/07/2019,4383.10,PV2019,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275018,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón
275019,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón
275020,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,GUERRERO,COCULA,COCULA,Cultivo de maíz grano,06/07/2019,2430.55,PV2019,guerrero-cocula,12,Guerrero,17,Cocula
275021,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,GUERRERO,ATLIXTAC,XALPITZÁHUAC,Cultivo de maíz grano,15/07/2019,6574.65,PV2019,guerrero-atlixtac,12,Guerrero,10,Atlixtac


In [33]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [34]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2019.csv', index=False)

In [35]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
KEY_benef_mun      0
CVE_ENT            0
Entidad_inegi      0
CVE_MUN            0
Municipio_inegi    0
dtype: int64

# 3. Join por entidad (Divide y Vencerás)

In [36]:
listado_beneficiarios_2019_match = pd.read_csv('../../data/listados_completos/listado_beneficiarios_2019_match.csv')

In [37]:
listado_beneficiarios_2019_match.columns

Index(['Unnamed: 0', 'BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO',
       'LOCALIDAD', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA'],
      dtype='object')

In [38]:
# Obtenemos las localidades únicas en el dataset.
listado_beneficiarios_2019_match = listado_beneficiarios_2019_match.drop(columns=['Unnamed: 0'])

In [39]:
listado_beneficiarios_2019_match['ENTIDAD_c_benef'] = listado_beneficiarios_2019_match['ENTIDAD'].apply(clean_text)
listado_beneficiarios_2019_match['MUNICIPIO_c_benef'] = listado_beneficiarios_2019_match['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_2019_match['LOCALIDAD_c_benef'] = listado_beneficiarios_2019_match['LOCALIDAD'].apply(clean_text)

In [40]:
listado_beneficiarios_2019_match['KEY_benef_loc'] = listado_beneficiarios_2019_match['MUNICIPIO_c_benef'].astype(str) + '-' + listado_beneficiarios_2019_match['LOCALIDAD_c_benef'].astype(str)

In [41]:
listado_beneficiarios_2019_match.shape

(276554, 13)

In [42]:
listado_beneficiarios_2019_match.ENTIDAD.unique()

array(['Puebla', 'Guerrero', 'México', 'Guanajuato', 'Querétaro',
       'Zacatecas', 'Veracruz de Ignacio de la Llave', 'Hidalgo',
       'Michoacán de Ocampo', 'Oaxaca', 'Colima', 'Chiapas', 'Jalisco',
       'San Luis Potosí', 'Nayarit', 'GUERRERO'], dtype=object)

In [43]:
listado_beneficiarios_2019_match[listado_beneficiarios_2019_match['LOCALIDAD'] == 'NACIONAL']

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,ENTIDAD_c_benef,MUNICIPIO_c_benef,LOCALIDAD_c_benef,KEY_benef_loc
0,HERNANDEZ MORALES BERNARDO,NACIONAL,Puebla,San Sebastián Tlacotepec,NACIONAL,Cultivo de maíz grano,2019,2191.55,PV2019,puebla,san sebastian tlacotepec,nacional,san sebastian tlacotepec-nacional
1,GUZMAN VARGAS CELEDONIA JUANA,NACIONAL,Guerrero,Malinaltepec,NACIONAL,Cultivo de maíz grano,2019,6813.65,PV2019,guerrero,malinaltepec,nacional,malinaltepec-nacional
2,REFUGIO RIVERA VALENTINA,NACIONAL,Guerrero,Malinaltepec,NACIONAL,Cultivo de maíz grano,2019,6813.65,PV2019,guerrero,malinaltepec,nacional,malinaltepec-nacional
3,LOPEZ GUZMAN AGAPITO,NACIONAL,Guerrero,Xochistlahuaca,NACIONAL,Cultivo de maíz grano,2019,4383.10,PV2019,guerrero,xochistlahuaca,nacional,xochistlahuaca-nacional
4,SALINAS SALGADO FELIPE,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2430.55,PV2019,guerrero,teloloapan,nacional,teloloapan-nacional
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,DAMASO TOMAS MARIA,NACIONAL,Guerrero,San Miguel Totolapan,NACIONAL,Cultivo de maíz grano,2019,4622.10,PV2019,guerrero,san miguel totolapan,nacional,san miguel totolapan-nacional
1527,HERRERA VILLANUEVA VALENTE,NACIONAL,Guerrero,Ayutla de los Libres,NACIONAL,Cultivo de maíz grano,2019,2307.60,PV2019,guerrero,ayutla de los libres,nacional,ayutla de los libres-nacional
1528,CRUZ NAJERA PERFECTO,NACIONAL,Guerrero,Atoyac de Álvarez,NACIONAL,Cultivo de maíz grano,2019,239.00,PV2019,guerrero,atoyac de alvarez,nacional,atoyac de alvarez-nacional
1529,CARRILLO VALENTE ISMAEL,NACIONAL,Guerrero,Tecoanapa,NACIONAL,Cultivo de maíz grano,2019,6574.65,PV2019,guerrero,tecoanapa,nacional,tecoanapa-nacional


### 3.2.1 Join de localidades - GUERRERO

Empezamos creando los datasets por entiedad del dataset de beneficiarios.


In [44]:
entities = [
    'Puebla', 'México', 'Guanajuato', 'Querétaro',
    'Zacatecas', 'Veracruz de Ignacio de la Llave', 'Hidalgo',
    'Michoacán de Ocampo', 'Oaxaca', 'Colima', 'Chiapas',
    'San Luis Potosí', 'Jalisco', 'Nayarit', 'GUERRERO'
]

for entity in entities:
    variable_name = f"Localidades_19_{entity.upper().replace(' ', '_')}"
    dataframe = globals()[variable_name] = listado_beneficiarios_2019_match[listado_beneficiarios_2019_match["ENTIDAD"] == entity]
    dataframe.drop_duplicates(subset='KEY_benef_loc', keep='first', inplace=True)
    print(f"{variable_name}: {dataframe.shape}")

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_7894/1770650823.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.drop_duplicates(subset='KEY_benef_loc', keep='first', inplace=True)


Localidades_19_PUEBLA: (3, 13)
Localidades_19_MÉXICO: (4, 13)
Localidades_19_GUANAJUATO: (1, 13)
Localidades_19_QUERÉTARO: (1, 13)
Localidades_19_ZACATECAS: (4, 13)
Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE: (1, 13)
Localidades_19_HIDALGO: (1, 13)
Localidades_19_MICHOACÁN_DE_OCAMPO: (1, 13)
Localidades_19_OAXACA: (5, 13)
Localidades_19_COLIMA: (1, 13)
Localidades_19_CHIAPAS: (2, 13)
Localidades_19_SAN_LUIS_POTOSÍ: (1, 13)
Localidades_19_JALISCO: (3, 13)
Localidades_19_NAYARIT: (1, 13)
Localidades_19_GUERRERO: (4576, 13)


In [45]:
Localidades_19_Guerrero = listado_beneficiarios_2019_match[listado_beneficiarios_2019_match['ENTIDAD'] == 'Guerrero']

In [46]:
Localidades_19_Guerrero.shape

(1499, 13)

### 3.2.2 Diccionarios de Inegi por entidad.

In [47]:
folder_path = '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/2019'

# Listar todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtrar solo los archivos CSV
csv_files = [file for file in files if file.endswith('.csv')]

# Leer cada archivo CSV y almacenar en variables separadas
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    # Crear un nombre de variable dinámicamente eliminando la extensión .csv
    var_name = file_name.replace('.csv', '')
    # Leer el archivo CSV y asignar el DataFrame a la variable
    globals()[var_name] = pd.read_csv(file_path)

# Opcional: Imprimir el nombre de las variables creadas y la cantidad de filas y columnas de cada DataFrame
for file_name in csv_files:
    var_name = file_name.replace('.csv', '')
    df = globals()[var_name]
    print(f'{var_name}: {df.shape[0]} filas, {df.shape[1]} columnas')


INEGI_UNIQUELOC_2019_jalisco: 18306 filas, 11 columnas
INEGI_UNIQUELOC_2019_colima: 2647 filas, 11 columnas
INEGI_UNIQUELOC_2019_guanajuato: 11130 filas, 11 columnas
INEGI_UNIQUELOC_2019_nayarit: 4474 filas, 11 columnas
INEGI_UNIQUELOC_2019_veracruz_de_ignacio_de_la_llave: 25305 filas, 11 columnas
INEGI_UNIQUELOC_2019_zacatecas: 6418 filas, 11 columnas
INEGI_UNIQUELOC_2019_michoacán_de_ocampo: 11119 filas, 11 columnas
INEGI_UNIQUELOC_2019_hidalgo: 5279 filas, 11 columnas
INEGI_UNIQUELOC_2019_chiapas: 21887 filas, 11 columnas
INEGI_UNIQUELOC_2019_querétaro: 2829 filas, 11 columnas
INEGI_UNIQUELOC_2019_méxico: 5217 filas, 11 columnas
INEGI_UNIQUELOC_2019_oaxaca: 12820 filas, 11 columnas
INEGI_UNIQUELOC_2019_san_luis_potosí: 8946 filas, 11 columnas
INEGI_UNIQUELOC_2019_guerrero: 9194 filas, 11 columnas
INEGI_UNIQUELOC_2019_puebla: 7708 filas, 11 columnas


In [48]:
INEGI_UNIQUELOC_2019_guerrero.shape

(9194, 11)

In [49]:
Localidades_19_GUERRERO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_GUERRERO, INEGI_UNIQUELOC_2019_guerrero, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_GUERRERO_parte_I.shape

Localidades_19_PUEBLA_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_PUEBLA, INEGI_UNIQUELOC_2019_puebla, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_PUEBLA_parte_I.shape

Localidades_19_MEXICO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_MÉXICO, INEGI_UNIQUELOC_2019_méxico, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_MEXICO_parte_I.shape

Localidades_19_GUANAJUATO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_GUANAJUATO, INEGI_UNIQUELOC_2019_guanajuato, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_GUANAJUATO_parte_I.shape

Localidades_19_QUERETARO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_QUERÉTARO, INEGI_UNIQUELOC_2019_querétaro, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_QUERETARO_parte_I.shape

Localidades_19_ZACATECAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_ZACATECAS, INEGI_UNIQUELOC_2019_zacatecas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_ZACATECAS_parte_I.shape

Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE, INEGI_UNIQUELOC_2019_veracruz_de_ignacio_de_la_llave, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE_parte_I.shape

Localidades_19_HIDALGO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_HIDALGO, INEGI_UNIQUELOC_2019_hidalgo, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_HIDALGO_parte_I.shape

Localidades_19_MICHOACAN_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_MICHOACÁN_DE_OCAMPO, INEGI_UNIQUELOC_2019_michoacán_de_ocampo, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_MICHOACAN_parte_I.shape

Localidades_19_OAXACA_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_OAXACA, INEGI_UNIQUELOC_2019_oaxaca, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_OAXACA_parte_I.shape

Localidades_19_COLIMA_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_COLIMA, INEGI_UNIQUELOC_2019_colima, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_COLIMA_parte_I.shape

Localidades_19_CHIAPAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_CHIAPAS, INEGI_UNIQUELOC_2019_chiapas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_CHIAPAS_parte_I.shape

Localidades_19_POTOSI_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_SAN_LUIS_POTOSÍ, INEGI_UNIQUELOC_2019_san_luis_potosí, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_POTOSI_parte_I.shape

Localidades_19_JALISCO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_JALISCO, INEGI_UNIQUELOC_2019_jalisco, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_JALISCO_parte_I.shape

Localidades_19_NAYARIT_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_NAYARIT, INEGI_UNIQUELOC_2019_nayarit, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_NAYARIT_parte_I.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_7894/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_7894/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_7894/734782849.py:18: SettingWithCopyWarning: 

(1, 26)

In [50]:
# Concatenate datasets vertically
diccionario_LOC_19 = pd.concat([
    Localidades_19_GUERRERO_parte_I,
    Localidades_19_PUEBLA_parte_I,
    Localidades_19_MEXICO_parte_I,
    Localidades_19_GUANAJUATO_parte_I,
    Localidades_19_QUERETARO_parte_I,
    Localidades_19_ZACATECAS_parte_I,
    Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE_parte_I,
    Localidades_19_HIDALGO_parte_I,
    Localidades_19_MICHOACAN_parte_I,
    Localidades_19_OAXACA_parte_I,
    Localidades_19_COLIMA_parte_I,
    Localidades_19_CHIAPAS_parte_I,
    Localidades_19_POTOSI_parte_I,
    Localidades_19_JALISCO_parte_I,
    Localidades_19_NAYARIT_parte_I
], axis=0)

# Check the shape of the concatenated DataFrame
diccionario_LOC_19.shape

(4605, 26)

In [51]:
diccionario_LOC_19.drop(columns=['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'ENTIDAD_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'best_match', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'CVE_LOC', 'Localidad_inegi', 'POB_TOTAL', 'Entidad_c_inegi',
       'Municipio_c_inegi', 'Localidad_c_inegi'], inplace=True)

In [52]:
diccionario_LOC_19.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_19.csv', index=False)

In [53]:
diccionario_LOC_19.isna().sum()

KEY_benef_loc          0
match_score            0
KEY_inegi_localidad    0
dtype: int64

# 4. Diccionarios de localidades y merge respectivos.

Empezamos leyendo el diccionario_LOC_20_simple, el cual solo tiene key_benef_mun y key_inegi_municipio manipulada.

In [54]:
import pandas as pd

# Intentar leer el archivo CSV con diferentes configuraciones de lectura
try:
    diccionario_LOC_19_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_19_simple.csv', encoding='cp1252', 
                          delimiter=';')
    #diccionario_LOC_21_simple.drop(columns=['ValidaciÃ³n', 'Propuesta'], inplace=True)
    print(diccionario_LOC_19_simple.head())
except pd.errors.ParserError as e:
    print("Error al leer el archivo CSV:", e)




                                    ï»¿KEY_benef_loc  match_score  \
0  teloloapan-cuadrilla del encanto (cerro del ba...         86.0   
1               arcelia-colonia emperador cuauhtemoc         86.0   
2  arcelia-reforma de lazaro cardenas (el aterriz...         86.0   
3                 marquelia-huerta luciano bonifacio         86.0   
4  atoyac de alvarez-colonia benito juarez (el po...         86.0   

                   KEY_inegi_localidad  
0                teloloapan-teloloapan  
1                      arcelia-arcelia  
2                      arcelia-arcelia  
3                  marquelia-marquelia  
4  atoyac de alvarez-atoyac de alvarez  


In [55]:
diccionario_LOC_19_simple.columns

Index(['ï»¿KEY_benef_loc', 'match_score', 'KEY_inegi_localidad'], dtype='object')

In [56]:
diccionario_LOC_19_simple.rename(columns={'ï»¿KEY_benef_loc': 'KEY_benef_loc'}, inplace=True)

In [57]:
diccionario_LOC_19_simple.columns

Index(['KEY_benef_loc', 'match_score', 'KEY_inegi_localidad'], dtype='object')

In [58]:
diccionario_LOC_19_simple

,KEY_benef_loc,match_score,KEY_inegi_localidad
0,teloloapan-cuadrilla del encanto (cerro del ba...,86.0,teloloapan-teloloapan
1,arcelia-colonia emperador cuauhtemoc,86.0,arcelia-arcelia
2,arcelia-reforma de lazaro cardenas (el aterriz...,86.0,arcelia-arcelia
3,marquelia-huerta luciano bonifacio,86.0,marquelia-marquelia
4,atoyac de alvarez-colonia benito juarez (el po...,86.0,atoyac de alvarez-atoyac de alvarez
...,...,...,...
4673,NaN,NaN,NaN
4674,NaN,NaN,NaN
4675,NaN,NaN,NaN
4676,NaN,NaN,NaN


In [59]:
listado_beneficiarios_2019_match.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'ENTIDAD_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'KEY_benef_loc'],
      dtype='object')

In [60]:
listado_beneficiarios_2019_match['MUNICIPIO_Clean'] = listado_beneficiarios_2019_match['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_2019_match['LOCALIDAD_Clean'] = listado_beneficiarios_2019_match['LOCALIDAD'].apply(clean_text)

# Create KEY in listado beneficiarios
listado_beneficiarios_2019_match['Municipio-loc-KEY'] = listado_beneficiarios_2019_match['MUNICIPIO_Clean'].astype(str) + '-' + listado_beneficiarios_2019_match[
    'LOCALIDAD_Clean'].astype(str)

Primer merge entre listado_beneficiarios_parte_II, y diccionario_LOC_22_simple. En este merge cogemos el dataset con los municipios corregidos previamente y ejecutamos el merge de manera que nos traemos las key_inegi_localidad corregidas.

In [61]:
listado_beneficiarios_2019_match.shape

(276554, 16)

In [62]:
INEGI_UNIQUELOC_2019 = dataset_inegi_2019.drop(columns=['Entidad_c_inegi', 'Municipio_c_inegi',
       'Localidad_c_inegi', 'POB_TOTAL'])
INEGI_UNIQUELOC_2019 = INEGI_UNIQUELOC_2019.drop_duplicates(subset='KEY_inegi_localidad', keep='first')

In [63]:
INEGI_UNIQUELOC_2019.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'KEY_inegi_municipio', 'KEY_inegi_localidad'],
      dtype='object')

In [64]:
INEGI_UNIQUELOC_2019

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,KEY_inegi_municipio,KEY_inegi_localidad
0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,aguascalientes-aguascalientes,aguascalientes-aguascalientes
1,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,aguascalientes-aguascalientes,aguascalientes-granja adelita
2,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,aguascalientes-aguascalientes,aguascalientes-agua azul
3,1,Aguascalientes,1,Aguascalientes,100,Rancho Alegre,aguascalientes-aguascalientes,aguascalientes-rancho alegre
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],aguascalientes-aguascalientes,aguascalientes-los arbolitos [rancho]
...,...,...,...,...,...,...,...,...
299565,32,Zacatecas,58,Santa María de la Paz,39,San Isidro,zacatecas-santa maria de la paz,santa maria de la paz-san isidro
299566,32,Zacatecas,58,Santa María de la Paz,40,San José,zacatecas-santa maria de la paz,santa maria de la paz-san jose
299567,32,Zacatecas,58,Santa María de la Paz,41,San Miguel Tepetitlán,zacatecas-santa maria de la paz,santa maria de la paz-san miguel tepetitlan
299568,32,Zacatecas,58,Santa María de la Paz,42,San Rafael,zacatecas-santa maria de la paz,santa maria de la paz-san rafael


In [65]:
listado_beneficiarios_2019_match = listado_beneficiarios_2019_match.drop_duplicates()

In [66]:
listado_beneficiarios_2019_match

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,ENTIDAD_c_benef,MUNICIPIO_c_benef,LOCALIDAD_c_benef,KEY_benef_loc,MUNICIPIO_Clean,LOCALIDAD_Clean,Municipio-loc-KEY
0,HERNANDEZ MORALES BERNARDO,NACIONAL,Puebla,San Sebastián Tlacotepec,NACIONAL,Cultivo de maíz grano,2019,2191.55,PV2019,puebla,san sebastian tlacotepec,nacional,san sebastian tlacotepec-nacional,san sebastian tlacotepec,nacional,san sebastian tlacotepec-nacional
1,GUZMAN VARGAS CELEDONIA JUANA,NACIONAL,Guerrero,Malinaltepec,NACIONAL,Cultivo de maíz grano,2019,6813.65,PV2019,guerrero,malinaltepec,nacional,malinaltepec-nacional,malinaltepec,nacional,malinaltepec-nacional
2,REFUGIO RIVERA VALENTINA,NACIONAL,Guerrero,Malinaltepec,NACIONAL,Cultivo de maíz grano,2019,6813.65,PV2019,guerrero,malinaltepec,nacional,malinaltepec-nacional,malinaltepec,nacional,malinaltepec-nacional
3,LOPEZ GUZMAN AGAPITO,NACIONAL,Guerrero,Xochistlahuaca,NACIONAL,Cultivo de maíz grano,2019,4383.10,PV2019,guerrero,xochistlahuaca,nacional,xochistlahuaca-nacional,xochistlahuaca,nacional,xochistlahuaca-nacional
4,SALINAS SALGADO FELIPE,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2430.55,PV2019,guerrero,teloloapan,nacional,teloloapan-nacional,teloloapan,nacional,teloloapan-nacional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276549,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero,taxco de alarcon,paintla,taxco de alarcon-paintla,taxco de alarcon,paintla,taxco de alarcon-paintla
276550,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero,taxco de alarcon,paintla,taxco de alarcon-paintla,taxco de alarcon,paintla,taxco de alarcon-paintla
276551,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,GUERRERO,COCULA,COCULA,Cultivo de maíz grano,06/07/2019,2430.55,PV2019,guerrero,cocula,cocula,cocula-cocula,cocula,cocula,cocula-cocula
276552,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,GUERRERO,ATLIXTAC,XALPITZÁHUAC,Cultivo de maíz grano,15/07/2019,6574.65,PV2019,guerrero,atlixtac,xalpitzahuac,atlixtac-xalpitzahuac,atlixtac,xalpitzahuac,atlixtac-xalpitzahuac


In [67]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I_localidades = pd.merge(listado_beneficiarios_2019_match, diccionario_LOC_19_simple, left_on="Municipio-loc-KEY",
                                         right_on="KEY_benef_loc", how='left', suffixes=('_benef', '_inegi'))

In [68]:
listado_beneficiarios_parte_I_localidades.shape

(276506, 19)

In [69]:
# Marcar todos los duplicados en las columnas 'columna1' y 'columna2' como True
duplicados = listado_beneficiarios_parte_I_localidades.duplicated(subset=['FECHA', 'BENEFICIARIO'], keep=False)

# Filtrar el DataFrame original para mostrar solo los duplicados
df_duplicados = listado_beneficiarios_parte_I_localidades[duplicados]

In [70]:
df_duplicados

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,ENTIDAD_c_benef,MUNICIPIO_c_benef,LOCALIDAD_c_benef,KEY_benef_loc_benef,MUNICIPIO_Clean,LOCALIDAD_Clean,Municipio-loc-KEY,KEY_benef_loc_inegi,match_score,KEY_inegi_localidad
1597,HERNANDEZ HERNANDEZ PEDRO,CENTRO PAÍS,GUERRERO,CHILAPA DE ÁLVAREZ,EL JAGÜEY,Cultivo de maíz grano,08/07/2019,2430.55,PV2019,guerrero,chilapa de alvarez,el jaguey,chilapa de alvarez-el jaguey,chilapa de alvarez,el jaguey,chilapa de alvarez-el jaguey,chilapa de alvarez-el jaguey,100.0,chilapa de alvarez-el jaguey
3382,SOTO SANCHEZ MARIA,CENTRO PAÍS,GUERRERO,AYUTLA DE LOS LIBRES,SAN JOSÉ LA HACIENDA,Cultivo de maíz grano,20/07/2019,2430.55,PV2019,guerrero,ayutla de los libres,san jose la hacienda,ayutla de los libres-san jose la hacienda,ayutla de los libres,san jose la hacienda,ayutla de los libres-san jose la hacienda,ayutla de los libres-san jose la hacienda,100.0,ayutla de los libres-san jose la hacienda
3685,DE LA CRUZ DE LA CRUZ GUADALUPE,CENTRO PAÍS,GUERRERO,OMETEPEC,COCHOAPA,Cultivo de maíz grano,15/07/2019,2430.55,PV2019,guerrero,ometepec,cochoapa,ometepec-cochoapa,ometepec,cochoapa,ometepec-cochoapa,ometepec-cochoapa,100.0,ometepec-cochoapa
4827,JIMENEZ RODRIGUEZ GABRIEL,CENTRO PAÍS,GUERRERO,QUECHULTENANGO,QUECHULTENANGO,Cultivo de maíz grano,09/07/2019,2430.55,PV2019,guerrero,quechultenango,quechultenango,quechultenango-quechultenango,quechultenango,quechultenango,quechultenango-quechultenango,quechultenango-quechultenango,100.0,quechultenango-quechultenango
5063,RAMIREZ MORA ANGEL,CENTRO PAÍS,GUERRERO,TECOANAPA,OCOTITLÁN,Cultivo de maíz grano,16/07/2019,4383.10,PV2019,guerrero,tecoanapa,ocotitlan,tecoanapa-ocotitlan,tecoanapa,ocotitlan,tecoanapa-ocotitlan,tecoanapa-ocotitlan,100.0,tecoanapa-ocotitlan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274974,GOMEZ HERNANDEZ MARIA,CENTRO PAÍS,GUERRERO,CHILAPA DE ÁLVAREZ,NEJAPA,Cultivo de maíz grano,08/07/2019,4622.10,PV2019,guerrero,chilapa de alvarez,nejapa,chilapa de alvarez-nejapa,chilapa de alvarez,nejapa,chilapa de alvarez-nejapa,chilapa de alvarez-nejapa,100.0,chilapa de alvarez-nejapa
275044,VALENTE MENESES ANADELI,CENTRO PAÍS,GUERRERO,TELOLOAPAN,MESAS NEBLINAS,Cultivo de maíz grano,21/07/2019,2191.55,PV2019,guerrero,teloloapan,mesas neblinas,teloloapan-mesas neblinas,teloloapan,mesas neblinas,teloloapan-mesas neblinas,teloloapan-mesas neblinas,100.0,teloloapan-mesas neblinas
275616,RAMIREZ PASCUAL TEOFILO,CENTRO PAÍS,GUERRERO,TELOLOAPAN,MESAS NEBLINAS,Cultivo de maíz grano,26/07/2019,4383.10,PV2019,guerrero,teloloapan,mesas neblinas,teloloapan-mesas neblinas,teloloapan,mesas neblinas,teloloapan-mesas neblinas,teloloapan-mesas neblinas,100.0,teloloapan-mesas neblinas
275654,BRITO MARTINEZ MARIA,CENTRO PAÍS,GUERRERO,ATOYAC DE ÁLVAREZ,EL CACAO,Cultivo de maíz grano,12/07/2019,2430.55,PV2019,guerrero,atoyac de alvarez,el cacao,atoyac de alvarez-el cacao,atoyac de alvarez,el cacao,atoyac de alvarez-el cacao,atoyac de alvarez-el cacao,100.0,atoyac de alvarez-el cacao


In [71]:
listado_beneficiarios_parte_I_localidades.drop_duplicates(keep='first', inplace=True)

In [72]:
listado_beneficiarios_parte_I_localidades.shape

(276506, 19)

In [73]:
listado_beneficiarios_parte_I_localidades.isna().sum()

BENEFICIARIO              0
ZONA                      0
ENTIDAD                   0
MUNICIPIO                 0
LOCALIDAD                 0
PRODUCTO                  0
FECHA                     0
MONTO FEDERAL             0
CICLO AGRÍCOLA            0
ENTIDAD_c_benef           0
MUNICIPIO_c_benef         0
LOCALIDAD_c_benef         0
KEY_benef_loc_benef       0
MUNICIPIO_Clean           0
LOCALIDAD_Clean           0
Municipio-loc-KEY         0
KEY_benef_loc_inegi       0
match_score            1488
KEY_inegi_localidad       0
dtype: int64

In [74]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_I_localidades[listado_beneficiarios_parte_I_localidades.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [75]:
rows_with_nan['KEY_benef_loc_benef'].unique()

array(['xochistlahuaca-nacional', 'teloloapan-nacional',
       'xalpatlahuac-nacional', 'san miguel totolapan-nacional',
       'jose joaquin de herrera-nacional', 'taxco de alarcon-nacional',
       'iguala de la independencia-nacional',
       'coyuca de catalan-nacional', 'ayutla de los libres-nacional',
       'atlixtac-nacional', 'quechultenango-nacional', 'arcelia-nacional',
       'alcozauca de guerrero-nacional',
       'ajuchitlan del progreso-nacional', 'acatepec-nacional',
       'zitlala-nacional', 'igualapa-nacional',
       'cutzamala de pinzon-nacional', 'olinala-nacional',
       'cochoapa el grande-nacional', 'petatlan-nacional',
       'tlalchapa-nacional', 'general heliodoro castillo-nacional',
       'tecoanapa-nacional', 'tixtla de guerrero-nacional',
       'cuetzala del progreso-nacional', 'azoyu-nacional',
       'juan r. escudero-nacional', 'martir de cuilapan-nacional',
       'ometepec-nacional', 'san marcos-nacional', 'copanatoyac-nacional',
       'la unio

In [76]:
print(listado_beneficiarios_parte_I_localidades['KEY_inegi_localidad'].duplicated().sum())
print(INEGI_UNIQUELOC_2019['KEY_inegi_localidad'].duplicated().sum())

271957
0


In [77]:
# TODO: REVISAR ESTA PARTE (ARTURO)
listado_beneficiarios_parte_II_localidades = pd.merge(listado_beneficiarios_parte_I_localidades, INEGI_UNIQUELOC_2019,
                                          left_on="KEY_inegi_localidad",
                                          right_on="KEY_inegi_localidad", how='left', suffixes=('_benef', '_inegi'))

In [78]:
listado_beneficiarios_parte_II_localidades.isna().sum()

BENEFICIARIO              0
ZONA                      0
ENTIDAD                   0
MUNICIPIO                 0
LOCALIDAD                 0
PRODUCTO                  0
FECHA                     0
MONTO FEDERAL             0
CICLO AGRÍCOLA            0
ENTIDAD_c_benef           0
MUNICIPIO_c_benef         0
LOCALIDAD_c_benef         0
KEY_benef_loc_benef       0
MUNICIPIO_Clean           0
LOCALIDAD_Clean           0
Municipio-loc-KEY         0
KEY_benef_loc_inegi       0
match_score            1488
KEY_inegi_localidad       0
CVE_ENT                   0
Entidad_inegi             0
CVE_MUN                   0
Municipio_inegi           0
CVE_LOC                   0
Localidad_inegi           0
KEY_inegi_municipio       0
dtype: int64

In [79]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_II_localidades[listado_beneficiarios_parte_II_localidades.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [80]:
rows_with_nan[rows_with_nan['Entidad_inegi'].isna()]

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,ENTIDAD_c_benef,...,KEY_benef_loc_inegi,match_score,KEY_inegi_localidad,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,KEY_inegi_municipio


In [81]:
rows_with_nan.tail(50)

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,ENTIDAD_c_benef,...,KEY_benef_loc_inegi,match_score,KEY_inegi_localidad,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,KEY_inegi_municipio
1479,SANDOVAL SALGADO FELIPE,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2430.55,PV2019,guerrero,...,teloloapan-nacional,NaN,teloloapan-teloloapan,12,Guerrero,58,Teloloapan,1,Teloloapan,guerrero-teloloapan
1480,ARTEAGA VELA INES,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2191.55,PV2019,guerrero,...,teloloapan-nacional,NaN,teloloapan-teloloapan,12,Guerrero,58,Teloloapan,1,Teloloapan,guerrero-teloloapan
1481,BASAN GUATEMALA CARMELO,NACIONAL,Guerrero,Tecoanapa,NACIONAL,Cultivo de maíz grano,2019,4383.10,PV2019,guerrero,...,tecoanapa-nacional,NaN,tecoanapa-tecoanapa,12,Guerrero,56,Tecoanapa,1,Tecoanapa,guerrero-tecoanapa
1482,ALVAREZ RABADAN VIRGINIA,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2430.55,PV2019,guerrero,...,teloloapan-nacional,NaN,teloloapan-teloloapan,12,Guerrero,58,Teloloapan,1,Teloloapan,guerrero-teloloapan
1483,PEREZ HERNANDEZ VIANEY,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2191.55,PV2019,guerrero,...,teloloapan-nacional,NaN,teloloapan-teloloapan,12,Guerrero,58,Teloloapan,1,Teloloapan,guerrero-teloloapan
1484,SALGADO MONTES DE OCA EZEQUIEL,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2430.55,PV2019,guerrero,...,teloloapan-nacional,NaN,teloloapan-teloloapan,12,Guerrero,58,Teloloapan,1,Teloloapan,guerrero-teloloapan
1485,MELGOZA SALMERON ESTHER VELIA,NACIONAL,Guerrero,Tecoanapa,NACIONAL,Cultivo de maíz grano,2019,4383.10,PV2019,guerrero,...,tecoanapa-nacional,NaN,tecoanapa-tecoanapa,12,Guerrero,56,Tecoanapa,1,Tecoanapa,guerrero-tecoanapa
1486,NAZARIO ONOFRE LUCILA,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2191.55,PV2019,guerrero,...,teloloapan-nacional,NaN,teloloapan-teloloapan,12,Guerrero,58,Teloloapan,1,Teloloapan,guerrero-teloloapan
1487,ROMAN SALGADO NELIDA,NACIONAL,Guerrero,Teloloapan,NACIONAL,Cultivo de maíz grano,2019,2430.55,PV2019,guerrero,...,teloloapan-nacional,NaN,teloloapan-teloloapan,12,Guerrero,58,Teloloapan,1,Teloloapan,guerrero-teloloapan
1488,RODRIGUEZ SAHAGUN LUIS,NACIONAL,Guerrero,San Miguel Totolapan,NACIONAL,Cultivo de maíz grano,2019,4383.10,PV2019,guerrero,...,san miguel totolapan-nacional,NaN,san miguel totolapan-san miguel totolapan,12,Guerrero,54,San Miguel Totolapan,1,San Miguel Totolapan,guerrero-san miguel totolapan


In [82]:
listado_beneficiarios_parte_II_localidades.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'ENTIDAD_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'KEY_benef_loc_benef',
       'MUNICIPIO_Clean', 'LOCALIDAD_Clean', 'Municipio-loc-KEY',
       'KEY_benef_loc_inegi', 'match_score', 'KEY_inegi_localidad', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'KEY_inegi_municipio'],
      dtype='object')

In [83]:
listado_beneficiarios_parte_II_localidades.drop(columns=['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'ENTIDAD_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'KEY_benef_loc_benef',
       'MUNICIPIO_Clean', 'LOCALIDAD_Clean', 'Municipio-loc-KEY',
       'KEY_benef_loc_inegi', 'match_score', 'KEY_inegi_localidad',
       'KEY_inegi_municipio'], inplace=True)

In [84]:
listado_beneficiarios_parte_II_localidades.columns

Index(['BENEFICIARIO', 'ZONA', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL',
       'CICLO AGRÍCOLA', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN',
       'Municipio_inegi', 'CVE_LOC', 'Localidad_inegi'],
      dtype='object')

In [85]:
listado_beneficiarios_parte_II_localidades.rename(columns={'BENEFICIARIO': 'Nombre del beneficiario',
                                                           'ZONA': 'Zona del país',
                                                           'CVE_ENT': 'Clave de entidad',
                                                            'Entidad_inegi': 'Entidad federativa',
                                                            'CVE_MUN': 'Clave de municipio',
                                                            'Municipio_inegi': 'Municipio',                                                           
                                                            'CVE_LOC': 'Clave de localidad',
                                                            'Localidad_inegi': 'Localidad',
                                                            'PRODUCTO': 'Producto cultivado',
                                                            'FECHA': 'Fecha de entrega del beneficio',
                                                            'MONTO FEDERAL': 'Monto entregado',
                                                            'CICLO AGRÍCOLA': 'Ciclo agrícola'}, inplace=True)

In [86]:
listado_beneficiarios_parte_II_localidades

,Nombre del beneficiario,Zona del país,Producto cultivado,Fecha de entrega del beneficio,Monto entregado,Ciclo agrícola,Clave de entidad,Entidad federativa,Clave de municipio,Municipio,Clave de localidad,Localidad
0,HERNANDEZ MORALES BERNARDO,NACIONAL,Cultivo de maíz grano,2019,2191.55,PV2019,21,Puebla,145,San Sebastián Tlacotepec,64,Xonoátl
1,GUZMAN VARGAS CELEDONIA JUANA,NACIONAL,Cultivo de maíz grano,2019,6813.65,PV2019,12,Guerrero,41,Malinaltepec,1,Malinaltepec
2,REFUGIO RIVERA VALENTINA,NACIONAL,Cultivo de maíz grano,2019,6813.65,PV2019,12,Guerrero,41,Malinaltepec,1,Malinaltepec
3,LOPEZ GUZMAN AGAPITO,NACIONAL,Cultivo de maíz grano,2019,4383.10,PV2019,12,Guerrero,71,Xochistlahuaca,1,Xochistlahuaca
4,SALINAS SALGADO FELIPE,NACIONAL,Cultivo de maíz grano,2019,2430.55,PV2019,12,Guerrero,58,Teloloapan,1,Teloloapan
...,...,...,...,...,...,...,...,...,...,...,...,...
276501,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,12,Guerrero,55,Taxco de Alarcón,26,Paintla
276502,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,12,Guerrero,55,Taxco de Alarcón,26,Paintla
276503,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,Cultivo de maíz grano,06/07/2019,2430.55,PV2019,12,Guerrero,17,Cocula,1,Cocula
276504,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,Cultivo de maíz grano,15/07/2019,6574.65,PV2019,12,Guerrero,10,Atlixtac,42,Xalpitzáhuac


In [87]:
listado_beneficiarios_parte_II_localidades.to_csv('../../data/listados_completos/listado_beneficiarios_2019_localidades.csv', index=False)